In [2]:
import json
import numpy as np
import os

In [3]:
DATA_DIR = '../data/'

with open(os.path.join(DATA_DIR, 'annotations', 'Human36M_subject1_data.json'), 'r') as jsonfile:
    data = json.load(jsonfile)
print(data.keys())

dict_keys(['images', 'annotations'])


In [8]:
def beautiful_json(data):
    return json.dumps(data, indent=4, sort_keys=True)

In [9]:
print(beautiful_json(data['images'][0]))

{
    "action_idx": 2,
    "action_name": "Directions",
    "cam_idx": 1,
    "file_name": "s_01_act_02_subact_01_ca_01/s_01_act_02_subact_01_ca_01_000001.jpg",
    "frame_idx": 0,
    "height": 1002,
    "id": 0,
    "subaction_idx": 1,
    "subject": 1,
    "width": 1000
}


In [12]:
print(len([x for x in data['images'] if x["action_idx"] == 2 ]))

11980
